**This script is used for consolidating data and perform feature engineering**

In [ ]:
## Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import sklearn

import os
import math

%matplotlib inline


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR="/content/drive/MyDrive/IAQF - Five+1 Guys/2022/Data"
LABEL_DIR = os.path.join(DATA_DIR,'Label')
DL_DIR =  "/content/drive/MyDrive/IAQF - Five+1 Guys/2022/Colab/Deep Learning"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import MLFramework

In [ ]:
%run '/content/drive/MyDrive/IAQF - Five+1 Guys/2022/Colab/MLFramework.ipynb' import MLFramework

# Consolidate Data version 4

In [ ]:
file_dir = os.path.join(LABEL_DIR,'master_ds_v3.csv')
df = pd.read_csv(file_dir,parse_dates=True,index_col=0)
df

,PX_LAST,PX_VOLUME,T1M,T3M,T6M,T1Y,T3Y,T5Y,T10Y,T30Y,T10Y3M,T10Y2Y,LIBOR_3M,TEDRATE,InterestRate,WTI_Futures_Price,WTI_Futures_Return,WTI_Spot_Price,WTI_Spot_Change,Commdity_channel_index_13,Williams_pct_14,RSI_14,Parabolic_0.02_0.2_0.02,DMI_14_ADX,Fear_Greedy_Index,CPI_monthly percent change,CPI_Release,Unemployment_rate,Unemployment_Release,change_in_total_nonfarm_thousand,Nonfarm_Release,VIX_Close,GDP_QOQ_change,GDP_Release,SPY_Adj_Close,SPY_Volume,SP_Adj_Close,SP_Volume,Mkt-RF,SMB,HML,RF,Russell3000_Adj_Close,Russell3000_Volume,POS_RET,POS_SR,State_RET_shifted,State_SR_shifted,hmm1,hmm1_MA5,hmm1_MA10,hmm1_MA15,hmm1_EWMA5,hmm1_EWMA10,hmm1_EWMA15,hmm1_shifted,hmm1_MA5_shifted,hmm1_MA10_shifted,hmm1_MA15_shifted,hmm1_EWMA5_shifted,hmm1_EWMA10_shifted,hmm1_EWMA15_shifted
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-02,585.41,NaN,NaN,7.83,7.50,7.81,7.90,7.87,7.94,8.00,0.11,0.07,8.38000,0.80,8.54,22.89,0.0490,22.89,0.0490,158.6294,-0.8405,69.0585,190.157,17.2194,2.3009,0.3,0.0,5.4,0.0,276.0,0.0,17.240000,0.8,0.0,NaN,NaN,359.69,1.620700e+08,1.44,-0.68,-0.07,0.026,200.309998,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,585.05,NaN,NaN,7.89,7.53,7.85,7.96,7.92,7.99,8.04,0.10,0.05,8.38000,0.75,8.37,23.68,0.0345,23.68,0.0345,166.5505,-6.7745,68.4778,190.767,18.4101,3.0652,0.3,0.0,5.4,0.0,276.0,0.0,18.190001,0.8,0.0,NaN,NaN,358.76,1.923300e+08,-0.06,0.74,-0.26,0.026,200.110001,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,580.97,NaN,NaN,7.84,7.50,7.82,7.93,7.91,7.98,8.04,0.14,0.06,8.38000,0.79,8.29,23.41,-0.0114,23.41,-0.0114,104.2355,-19.9820,61.8875,191.575,18.1196,1.8271,0.3,0.0,5.4,0.0,276.0,0.0,19.219999,0.8,0.0,NaN,NaN,355.67,1.770000e+08,-0.71,0.42,-0.22,0.026,198.850006,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,575.74,NaN,NaN,7.79,7.46,7.79,7.94,7.92,7.99,8.06,0.20,0.09,8.38000,0.84,8.20,23.08,-0.0141,23.08,-0.0141,56.8323,-36.1185,54.9317,192.319,17.4400,-0.2747,0.3,0.0,5.4,0.0,84.0,1.0,20.110001,0.8,0.0,NaN,NaN,352.20,1.585300e+08,-0.85,0.77,-0.21,0.026,196.860001,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,577.78,NaN,NaN,7.79,7.49,7.81,7.95,7.92,8.02,8.09,0.23,0.12,8.38000,0.84,8.27,21.62,-0.0633,21.62,-0.0633,43.9596,-30.0539,56.8927,193.003,16.6146,-0.5619,0.3,0.0,5.4,0.0,84.0,0.0,20.260000,0.8,0.0,NaN,NaN,353.79,1.401100e+08,0.30,-0.41,-0.26,0.026,197.539993,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,14547.22,1.322548e+09,0.05,0.17,0.36,0.57,1.33,1.62,1.83,2.14,1.66,0.79,0.25514,0.09,0.08,86.96,0.0179,85.58,-0.0135,-167.3056,-99.3827,32.9727,2780.178,21.6253,-124.8524,0.5,0.0,3.9,0.0,199.0,0.0,23.850000,2.3,0.0,451.750000,109357600.0,4532.76,3.387370e+09,-0.31,0.06,0.53,0.000,2633.860107,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0
2022-01-20,14378.11,1.355285e+09,0.05,0.17,0.36,0.60,1.34,1.62,1.83,2.14,1.66,0.75,0.25886,0.08,0.08,86.90,-0.0007,84.67,-0.0106,-163.7336,-98.8126,29.7787,2768.382,23.3995,-171.7405,0.5,0.0,3.9,0.0,199.0,0.0,25.590000,2.3,0.0,446.750000,122379700.0,4482.73,3.473060e+09,-0.31,0.06,0.53,0.000,2603.060059,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0
2022-01-21,14099.17,1.917342e+09,0.05,0.17,0.35,0.58,1.28,1.54,1.75,2.07,1.58,0.74,0.25771,0.09,0.08,85.14,-0.0203,85.43,0.0090,-191.9934,-99.7294,25.4267,2751.593,25.6026,-213.5388,0.5,0.0,3.9,0.0,199.0,0.0,28.850000,2.3,0.0,437.980011,202271200.0,4397.94,3.945810e+09,-0.31,0.06,0.53,0.000,2552.540039,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.

In [ ]:
df = df.loc[:,~df.columns.str.contains('hmm')]

## Merge with new hmm file


In [ ]:
hmm_dir = os.path.join(LABEL_DIR,'label by hmm v3.csv')
hmm = pd.read_csv(hmm_dir,parse_dates=True,index_col=0)
hmm

,log_ret,hmm1,hmm1_MA5,hmm1_MA10,hmm1_MA15,hmm1_EWMA5,hmm1_EWMA10,hmm1_EWMA15,hmm1_duration_bear,hmm1_duration_static,hmm1_duration_bull,hmm1_MA5_duration_bear,hmm1_MA5_duration_static,hmm1_MA5_duration_bull,hmm1_MA10_duration_bear,hmm1_MA10_duration_static,hmm1_MA10_duration_bull,hmm1_MA15_duration_bear,hmm1_MA15_duration_static,hmm1_MA15_duration_bull,hmm1_EWMA5_duration_bear,hmm1_EWMA5_duration_static,hmm1_EWMA5_duration_bull,hmm1_EWMA10_duration_bear,hmm1_EWMA10_duration_static,hmm1_EWMA10_duration_bull,hmm1_EWMA15_duration_bear,hmm1_EWMA15_duration_static,hmm1_EWMA15_duration_bull,hmm2,hmm2_MA5,hmm2_MA10,hmm2_MA15,hmm2_EWMA5,hmm2_EWMA10,hmm2_EWMA15,hmm2_duration_bear,hmm2_duration_static,hmm2_duration_bull,hmm2_MA5_duration_bear,hmm2_MA5_duration_static,hmm2_MA5_duration_bull,hmm2_MA10_duration_bear,hmm2_MA10_duration_static,hmm2_MA10_duration_bull,hmm2_MA15_duration_bear,hmm2_MA15_duration_static,hmm2_MA15_duration_bull,hmm2_EWMA5_duration_bear,hmm2_EWMA5_duration_static,hmm2_EWMA5_duration_bull,hmm2_EWMA10_duration_bear,hmm2_EWMA10_duration_static,hmm2_EWMA10_duration_bull,hmm2_EWMA15_duration_bear,hmm2_EWMA15_duration_static,hmm2_EWMA15_duration_bull
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-03,-0.009003,1,0,1,1,1,1,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0
2000-01-04,-0.039434,-1,-1,1,1,-1,-1,-1,1,0,0,1,0,0,0,0,2,0,0,2,1,0,0,1,0,0,1,0,0,-1,-1,-1,-1,-1,-1,-1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
2000-01-05,0.001705,-1,-1,1,1,-1,-1,-1,2,0,0,2,0,0,0,0,3,0,0,3,2,0,0,2,0,0,2,0,0,-1,-1,-1,-1,-1,-1,-1,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0
2000-01-06,-0.000476,-1,-1,1,1,-1,-1,-1,3,0,0,3,0,0,0,0,4,0,0,4,3,0,0,3,0,0,3,0,0,-1,-1,-1,-1,-1,-1,-1,3,0,0,3,0,0,3,0,0,3,0,0,3,0,0,3,0,0,3,0,0
2000-01-07,0.026756,-1,-1,1,1,-1,0,0,4,0,0,4,0,0,0,0,5,0,0,5,4,0,0,0,1,0,0,1,0,-1,0,0,0,-1,0,0,4,0,0,0,1,0,0,1,0,0,1,0,4,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,-0.010180,0,-1,-1,-1,-1,-1,-1,0,2,0,2,0,0,4,0,0,2,0,0,2,0,0,4,0,0,3,0,0,-1,-1,-1,-1,-1,-1,-1,2,0,0,2,0,0,4,0,0,4,0,0,2,0,0,5,0,0,10,0,0
2022-01-20,-0.011763,0,-1,-1,-1,-1,-1,-1,0,3,0,3,0,0,5,0,0,3,0,0,3,0,0,5,0,0,4,0,0,-1,-1,-1,-1,-1,-1,-1,3,0,0,3,0,0,5,0,0,5,0,0,3,0,0,6,0,0,11,0,0
2022-01-21,-0.019599,-1,-1,-1,-1,-1,-1,-1,1,0,0,4,0,0,6,0,0,4,0,0,4,0,0,6,0,0,5,0,0,-1,-1,-1,-1,-1,-1,-1,4,0,0,4,0,0,6,0,0,6,0,0,4,0,0,7,0,0,12,0,0


In [ ]:
hmm.drop('log_ret',axis=1,inplace=True)
df = df.merge(hmm,how='left',left_index=True,right_index=True,sort=True)
df

,PX_LAST,PX_VOLUME,T1M,T3M,T6M,T1Y,T3Y,T5Y,T10Y,T30Y,T10Y3M,T10Y2Y,LIBOR_3M,TEDRATE,InterestRate,WTI_Futures_Price,WTI_Futures_Return,WTI_Spot_Price,WTI_Spot_Change,Commdity_channel_index_13,Williams_pct_14,RSI_14,Parabolic_0.02_0.2_0.02,DMI_14_ADX,Fear_Greedy_Index,CPI_monthly percent change,CPI_Release,Unemployment_rate,Unemployment_Release,change_in_total_nonfarm_thousand,Nonfarm_Release,VIX_Close,GDP_QOQ_change,GDP_Release,SPY_Adj_Close,SPY_Volume,SP_Adj_Close,SP_Volume,Mkt-RF,SMB,...,hmm1_MA15_duration_bear,hmm1_MA15_duration_static,hmm1_MA15_duration_bull,hmm1_EWMA5_duration_bear,hmm1_EWMA5_duration_static,hmm1_EWMA5_duration_bull,hmm1_EWMA10_duration_bear,hmm1_EWMA10_duration_static,hmm1_EWMA10_duration_bull,hmm1_EWMA15_duration_bear,hmm1_EWMA15_duration_static,hmm1_EWMA15_duration_bull,hmm2,hmm2_MA5,hmm2_MA10,hmm2_MA15,hmm2_EWMA5,hmm2_EWMA10,hmm2_EWMA15,hmm2_duration_bear,hmm2_duration_static,hmm2_duration_bull,hmm2_MA5_duration_bear,hmm2_MA5_duration_static,hmm2_MA5_duration_bull,hmm2_MA10_duration_bear,hmm2_MA10_duration_static,hmm2_MA10_duration_bull,hmm2_MA15_duration_bear,hmm2_MA15_duration_static,hmm2_MA15_duration_bull,hmm2_EWMA5_duration_bear,hmm2_EWMA5_duration_static,hmm2_EWMA5_duration_bull,hmm2_EWMA10_duration_bear,hmm2_EWMA10_duration_static,hmm2_EWMA10_duration_bull,hmm2_EWMA15_duration_bear,hmm2_EWMA15_duration_static,hmm2_EWMA15_duration_bull
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-02,585.41,NaN,NaN,7.83,7.50,7.81,7.90,7.87,7.94,8.00,0.11,0.07,8.38000,0.80,8.54,22.89,0.0490,22.89,0.0490,158.6294,-0.8405,69.0585,190.157,17.2194,2.3009,0.3,0.0,5.4,0.0,276.0,0.0,17.240000,0.8,0.0,NaN,NaN,359.69,1.620700e+08,1.44,-0.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,585.05,NaN,NaN,7.89,7.53,7.85,7.96,7.92,7.99,8.04,0.10,0.05,8.38000,0.75,8.37,23.68,0.0345,23.68,0.0345,166.5505,-6.7745,68.4778,190.767,18.4101,3.0652,0.3,0.0,5.4,0.0,276.0,0.0,18.190001,0.8,0.0,NaN,NaN,358.76,1.923300e+08,-0.06,0.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,580.97,NaN,NaN,7.84,7.50,7.82,7.93,7.91,7.98,8.04,0.14,0.06,8.38000,0.79,8.29,23.41,-0.0114,23.41,-0.0114,104.2355,-19.9820,61.8875,191.575,18.1196,1.8271,0.3,0.0,5.4,0.0,276.0,0.0,19.219999,0.8,0.0,NaN,NaN,355.67,1.770000e+08,-0.71,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,575.74,NaN,NaN,7.79,7.46,7.79,7.94,7.92,7.99,8.06,0.20,0.09,8.38000,0.84,8.20,23.08,-0.0141,23.08,-0.0141,56.8323,-36.1185,54.9317,192.319,17.4400,-0.2747,0.3,0.0,5.4,0.0,84.0,1.0,20.110001,0.8,0.0,NaN,NaN,352.20,1.585300e+08,-0.85,0.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,577.78,NaN,NaN,7.79,7.49,7.81,7.95,7.92,8.02,8.09,0.23,0.12,8.38000,0.84,8.27,21.62,-0.0633,21.62,-0.0633,43.9596,-30.0539,56.8927,193.003,16.6146,-0.5619,0.3,0.0,5.4,0.0,84.0,0.0,20.260000,0.8,0.0,NaN,NaN,353.79,1.401100e+08,0.30,-0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,14547.22,1.322548e+09,0.05,0.17,0.36,0.57,1.33,1.62,1.83,2.14,1.66,0.79,0.25514,0.09,0.08,86.96,0.0179,85.58,-0.0135,-167.3056,-9

## Shift hmm labels

In [ ]:
hmm_label = ['hmm1','hmm1_MA5','hmm1_MA10','hmm1_MA15','hmm1_EWMA5','hmm1_EWMA10','hmm1_EWMA15','hmm2','hmm2_MA5','hmm2_MA10','hmm2_MA15','hmm2_EWMA5','hmm2_EWMA10','hmm2_EWMA15']
for hmm in hmm_label:
  df.loc[:,hmm+'_shifted'] = df[hmm].shift(-1)
df

,PX_LAST,PX_VOLUME,T1M,T3M,T6M,T1Y,T3Y,T5Y,T10Y,T30Y,T10Y3M,T10Y2Y,LIBOR_3M,TEDRATE,InterestRate,WTI_Futures_Price,WTI_Futures_Return,WTI_Spot_Price,WTI_Spot_Change,Commdity_channel_index_13,Williams_pct_14,RSI_14,Parabolic_0.02_0.2_0.02,DMI_14_ADX,Fear_Greedy_Index,CPI_monthly percent change,CPI_Release,Unemployment_rate,Unemployment_Release,change_in_total_nonfarm_thousand,Nonfarm_Release,VIX_Close,GDP_QOQ_change,GDP_Release,SPY_Adj_Close,SPY_Volume,SP_Adj_Close,SP_Volume,Mkt-RF,SMB,...,hmm2_MA10,hmm2_MA15,hmm2_EWMA5,hmm2_EWMA10,hmm2_EWMA15,hmm2_duration_bear,hmm2_duration_static,hmm2_duration_bull,hmm2_MA5_duration_bear,hmm2_MA5_duration_static,hmm2_MA5_duration_bull,hmm2_MA10_duration_bear,hmm2_MA10_duration_static,hmm2_MA10_duration_bull,hmm2_MA15_duration_bear,hmm2_MA15_duration_static,hmm2_MA15_duration_bull,hmm2_EWMA5_duration_bear,hmm2_EWMA5_duration_static,hmm2_EWMA5_duration_bull,hmm2_EWMA10_duration_bear,hmm2_EWMA10_duration_static,hmm2_EWMA10_duration_bull,hmm2_EWMA15_duration_bear,hmm2_EWMA15_duration_static,hmm2_EWMA15_duration_bull,hmm1_shifted,hmm1_MA5_shifted,hmm1_MA10_shifted,hmm1_MA15_shifted,hmm1_EWMA5_shifted,hmm1_EWMA10_shifted,hmm1_EWMA15_shifted,hmm2_shifted,hmm2_MA5_shifted,hmm2_MA10_shifted,hmm2_MA15_shifted,hmm2_EWMA5_shifted,hmm2_EWMA10_shifted,hmm2_EWMA15_shifted
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-02,585.41,NaN,NaN,7.83,7.50,7.81,7.90,7.87,7.94,8.00,0.11,0.07,8.38000,0.80,8.54,22.89,0.0490,22.89,0.0490,158.6294,-0.8405,69.0585,190.157,17.2194,2.3009,0.3,0.0,5.4,0.0,276.0,0.0,17.240000,0.8,0.0,NaN,NaN,359.69,1.620700e+08,1.44,-0.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,585.05,NaN,NaN,7.89,7.53,7.85,7.96,7.92,7.99,8.04,0.10,0.05,8.38000,0.75,8.37,23.68,0.0345,23.68,0.0345,166.5505,-6.7745,68.4778,190.767,18.4101,3.0652,0.3,0.0,5.4,0.0,276.0,0.0,18.190001,0.8,0.0,NaN,NaN,358.76,1.923300e+08,-0.06,0.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,580.97,NaN,NaN,7.84,7.50,7.82,7.93,7.91,7.98,8.04,0.14,0.06,8.38000,0.79,8.29,23.41,-0.0114,23.41,-0.0114,104.2355,-19.9820,61.8875,191.575,18.1196,1.8271,0.3,0.0,5.4,0.0,276.0,0.0,19.219999,0.8,0.0,NaN,NaN,355.67,1.770000e+08,-0.71,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,575.74,NaN,NaN,7.79,7.46,7.79,7.94,7.92,7.99,8.06,0.20,0.09,8.38000,0.84,8.20,23.08,-0.0141,23.08,-0.0141,56.8323,-36.1185,54.9317,192.319,17.4400,-0.2747,0.3,0.0,5.4,0.0,84.0,1.0,20.110001,0.8,0.0,NaN,NaN,352.20,1.585300e+08,-0.85,0.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,577.78,NaN,NaN,7.79,7.49,7.81,7.95,7.92,8.02,8.09,0.23,0.12,8.38000,0.84,8.27,21.62,-0.0633,21.62,-0.0633,43.9596,-30.0539,56.8927,193.003,16.6146,-0.5619,0.3,0.0,5.4,0.0,84.0,0.0,20.260000,0.8,0.0,NaN,NaN,353.79,1.401100e+08,0.30,-0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,14547.22,1.322548e+09,0.05,0.17,0.36,0.57,1.33,1.62,1.83,2.14,1.66,0.79,0.25514,0.09,0.08,86.96,0.0179,85.58,-0.0135,-167.3056,-99.3827,32.9727,2780.178,21.6253,-124.8524,0.5,0.0,3.9,0.0,199.0,0.0,23.8500

## Features Engineering

In [ ]:
# Add Weekday
weekday =df.index.map(lambda x: x.weekday()).values.reshape(-1,1)
OHE = OneHotEncoder(drop='first',sparse=False)
weekday_OHE = OHE.fit_transform(weekday)
temp = pd.DataFrame(weekday_OHE,index=df.index, columns=OHE.get_feature_names_out(['weekday']))
df = df.merge(temp,left_index=True,right_index=True,)

In [ ]:
# Add Month
month = df.index.map(lambda x: x.month).values.reshape(-1,1)
OHE = OneHotEncoder(drop='first',sparse=False)
month_OHE = OHE.fit_transform(month)
temp = pd.DataFrame(month_OHE,index=df.index, columns=OHE.get_feature_names_out(['month']))
df = df.merge(temp,left_index=True,right_index=True)

In [ ]:
df

,PX_LAST,PX_VOLUME,T1M,T3M,T6M,T1Y,T3Y,T5Y,T10Y,T30Y,T10Y3M,T10Y2Y,LIBOR_3M,TEDRATE,InterestRate,WTI_Futures_Price,WTI_Futures_Return,WTI_Spot_Price,WTI_Spot_Change,Commdity_channel_index_13,Williams_pct_14,RSI_14,Parabolic_0.02_0.2_0.02,DMI_14_ADX,Fear_Greedy_Index,CPI_monthly percent change,CPI_Release,Unemployment_rate,Unemployment_Release,change_in_total_nonfarm_thousand,Nonfarm_Release,VIX_Close,GDP_QOQ_change,GDP_Release,SPY_Adj_Close,SPY_Volume,SP_Adj_Close,SP_Volume,Mkt-RF,SMB,...,hmm2_MA15_duration_static,hmm2_MA15_duration_bull,hmm2_EWMA5_duration_bear,hmm2_EWMA5_duration_static,hmm2_EWMA5_duration_bull,hmm2_EWMA10_duration_bear,hmm2_EWMA10_duration_static,hmm2_EWMA10_duration_bull,hmm2_EWMA15_duration_bear,hmm2_EWMA15_duration_static,hmm2_EWMA15_duration_bull,hmm1_shifted,hmm1_MA5_shifted,hmm1_MA10_shifted,hmm1_MA15_shifted,hmm1_EWMA5_shifted,hmm1_EWMA10_shifted,hmm1_EWMA15_shifted,hmm2_shifted,hmm2_MA5_shifted,hmm2_MA10_shifted,hmm2_MA15_shifted,hmm2_EWMA5_shifted,hmm2_EWMA10_shifted,hmm2_EWMA15_shifted,weekday_1,weekday_2,weekday_3,weekday_4,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-02,585.41,NaN,NaN,7.83,7.50,7.81,7.90,7.87,7.94,8.00,0.11,0.07,8.38000,0.80,8.54,22.89,0.0490,22.89,0.0490,158.6294,-0.8405,69.0585,190.157,17.2194,2.3009,0.3,0.0,5.4,0.0,276.0,0.0,17.240000,0.8,0.0,NaN,NaN,359.69,1.620700e+08,1.44,-0.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-03,585.05,NaN,NaN,7.89,7.53,7.85,7.96,7.92,7.99,8.04,0.10,0.05,8.38000,0.75,8.37,23.68,0.0345,23.68,0.0345,166.5505,-6.7745,68.4778,190.767,18.4101,3.0652,0.3,0.0,5.4,0.0,276.0,0.0,18.190001,0.8,0.0,NaN,NaN,358.76,1.923300e+08,-0.06,0.74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-04,580.97,NaN,NaN,7.84,7.50,7.82,7.93,7.91,7.98,8.04,0.14,0.06,8.38000,0.79,8.29,23.41,-0.0114,23.41,-0.0114,104.2355,-19.9820,61.8875,191.575,18.1196,1.8271,0.3,0.0,5.4,0.0,276.0,0.0,19.219999,0.8,0.0,NaN,NaN,355.67,1.770000e+08,-0.71,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-05,575.74,NaN,NaN,7.79,7.46,7.79,7.94,7.92,7.99,8.06,0.20,0.09,8.38000,0.84,8.20,23.08,-0.0141,23.08,-0.0141,56.8323,-36.1185,54.9317,192.319,17.4400,-0.2747,0.3,0.0,5.4,0.0,84.0,1.0,20.110001,0.8,0.0,NaN,NaN,352.20,1.585300e+08,-0.85,0.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-08,577.78,NaN,NaN,7.79,7.49,7.81,7.95,7.92,8.02,8.09,0.23,0.12,8.38000,0.84,8.27,21.62,-0.0633,21.62,-0.0633,43.9596,-30.0539,56.8927,193.003,16.6146,-0.5619,0.3,0.0,5.4,0.0,84.0,0.0,20.260000,0.8,0.0,NaN,NaN,353.79,1.401100e+08,0.30,-0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,14547.22,1.322548e+09,0.05,0.17,0.36,0.57,1.33,1.62,1.83,2.14,1.66,0.79,0.25514,0.09,0.08,86.96,0.0179,85.58,-0.0135,-167.3056,-99.3827,32.9727,2780.178,21.6253,-124.8524,0.5,0.0,3.9,0.0,199.0,0.0,23.850000,2.3,0.0,451.750000,109357600.0,4532.76,3.387370e+09,-0.31,0.06,...,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,10.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.

In [ ]:
df.to_csv(os.path.join(LABEL_DIR,'tempMasterds.csv'))

# Return

In [ ]:
# subcolumns = ['PX_LAST','SPY_Adj_Close','SP_Adj_Close']
# for c in subcolumns:
#     df.loc[:,c+'_Ret'] = np.log(df.loc[:,c] / df.loc[:,c].shift(1) ) 

df.loc[:,'PX_Ret'] = np.log(df.loc[:,'PX_LAST'] / df.loc[:,'PX_LAST'].shift(1) ) 
# df.loc[:,'SPTI_Ret'] = np.log(df.loc[:,'SPTI_Adj_Close'] / df.loc[:,'SPTI_Adj_Close'].shift(1) ) 


## Differencing

In [ ]:
subcolumns = ['VIX_Close','Mkt-RF','SMB','HML','RF','T1M','T3M','T6M','T1Y','T3Y','T5Y','T10Y','T30Y','T10Y3M','T10Y2Y','LIBOR_3M','TEDRATE','InterestRate']
for c in subcolumns:
    df.loc[:,c+'_Diff'] = df.loc[:,c] - df.loc[:,c].shift(1) 

In [ ]:
df

,PX_LAST,PX_VOLUME,T1M,T3M,T6M,T1Y,T3Y,T5Y,T10Y,T30Y,T10Y3M,T10Y2Y,LIBOR_3M,TEDRATE,InterestRate,WTI_Futures_Price,WTI_Futures_Return,WTI_Spot_Price,WTI_Spot_Change,Commdity_channel_index_13,Williams_pct_14,RSI_14,Parabolic_0.02_0.2_0.02,DMI_14_ADX,Fear_Greedy_Index,CPI_monthly percent change,CPI_Release,Unemployment_rate,Unemployment_Release,change_in_total_nonfarm_thousand,Nonfarm_Release,VIX_Close,GDP_QOQ_change,GDP_Release,SPY_Adj_Close,SPY_Volume,SP_Adj_Close,SP_Volume,Mkt-RF,SMB,...,hmm2_MA5_shifted,hmm2_MA10_shifted,hmm2_MA15_shifted,hmm2_EWMA5_shifted,hmm2_EWMA10_shifted,hmm2_EWMA15_shifted,weekday_1,weekday_2,weekday_3,weekday_4,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,PX_Ret,VIX_Close_Diff,Mkt-RF_Diff,SMB_Diff,HML_Diff,RF_Diff,T1M_Diff,T3M_Diff,T6M_Diff,T1Y_Diff,T3Y_Diff,T5Y_Diff,T10Y_Diff,T30Y_Diff,T10Y3M_Diff,T10Y2Y_Diff,LIBOR_3M_Diff,TEDRATE_Diff,InterestRate_Diff
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-02,585.41,NaN,NaN,7.83,7.50,7.81,7.90,7.87,7.94,8.00,0.11,0.07,8.38000,0.80,8.54,22.89,0.0490,22.89,0.0490,158.6294,-0.8405,69.0585,190.157,17.2194,2.3009,0.3,0.0,5.4,0.0,276.0,0.0,17.240000,0.8,0.0,NaN,NaN,359.69,1.620700e+08,1.44,-0.68,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,585.05,NaN,NaN,7.89,7.53,7.85,7.96,7.92,7.99,8.04,0.10,0.05,8.38000,0.75,8.37,23.68,0.0345,23.68,0.0345,166.5505,-6.7745,68.4778,190.767,18.4101,3.0652,0.3,0.0,5.4,0.0,276.0,0.0,18.190001,0.8,0.0,NaN,NaN,358.76,1.923300e+08,-0.06,0.74,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000615,0.950001,-1.50,1.42,-0.19,0.0,NaN,0.06,0.03,0.04,0.06,0.05,0.05,0.04,-0.01,-0.02,0.00000,-0.05,-0.17
1990-01-04,580.97,NaN,NaN,7.84,7.50,7.82,7.93,7.91,7.98,8.04,0.14,0.06,8.38000,0.79,8.29,23.41,-0.0114,23.41,-0.0114,104.2355,-19.9820,61.8875,191.575,18.1196,1.8271,0.3,0.0,5.4,0.0,276.0,0.0,19.219999,0.8,0.0,NaN,NaN,355.67,1.770000e+08,-0.71,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006998,1.029998,-0.65,-0.32,0.04,0.0,NaN,-0.05,-0.03,-0.03,-0.03,-0.01,-0.01,0.00,0.04,0.01,0.00000,0.04,-0.08
1990-01-05,575.74,NaN,NaN,7.79,7.46,7.79,7.94,7.92,7.99,8.06,0.20,0.09,8.38000,0.84,8.20,23.08,-0.0141,23.08,-0.0141,56.8323,-36.1185,54.9317,192.319,17.4400,-0.2747,0.3,0.0,5.4,0.0,84.0,1.0,20.110001,0.8,0.0,NaN,NaN,352.20,1.585300e+08,-0.85,0.77,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.009043,0.890002,-0.14,0.35,0.01,0.0,NaN,-0.05,-0.04,-0.03,0.01,0.01,0.01,0.02,0.06,0.03,0.00000,0.05,-0.09
1990-01-08,577.78,NaN,NaN,7.79,7.49,7.81,7.95,7.92,8.02,8.09,0.23,0.12,8.38000,0.84,8.27,21.62,-0.0633,21.62,-0.0633,43.9596,-30.0539,56.8927,193.003,16.6146,-0.5619,0.3,0.0,5.4,0.0,84.0,0.0,20.260000,0.8,0.0,NaN,NaN,353.79,1.401100e+08,0.30,-0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003537,0.149999,1.15,-1.18,-0.05,0.0,NaN,0.00,0.03,0.02,0.01,0.00,0.03,0.03,0.03,0.03,0.00000,0.00,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,14547.22,1.322548e+09,0.05,0.17,0.36,0.57,1.33,1.62,1.83,2.14,1.66,0.79,0.25514,0.09,0.08,86.96,0.0179,85.58,-0.0135,-167.3056,-99.3827,32.9727,2780.178,21.6253,-124.8524,0.5,0.0,3.9,0.0,199.0,0.0,23.850000,2.3,0.0,451.750000,109357600.0,4532.76,3.387370e+09,-0.31,0.06,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.010162,1.059999,0.00,0.00,0.00,0.0,0.0,0.01,-0.01,-0.01,-0.02,-0.03,-0.04,

## Technical Indicator (Use EMA5 to calculate TI)

In [ ]:
!pip install ta
import ta

  Created wheel for ta: filename=ta-0.9.0-py3-none-any.whl size=28908 sha256=d22fa327a9de42ecc345c440e94d8902b37f65d04e0f9e81e6f6f023a2e0847f
  Stored in directory: /root/.cache/pip/wheels/72/78/64/cc1c01506a1010a9845e9bd7c69333730f7174661228ea4f98
Successfully built ta


In [ ]:
!pip install plotly

import plotly


### EMA

In [ ]:
df.loc[:,'PX_EMA5'] = ta.trend.ema_indicator(df['PX_LAST'], window=5, fillna=False)
df.loc[:,'PX_KAMA10'] = ta.momentum.kama(close = df['PX_LAST'], window=10, pow1=2, pow2=30, fillna=False)
# ta.trend.ema_indicator(df['PX_LAST'], window=10, fillna=False)

close = df['PX_EMA5']
volume = df['PX_VOLUME']

In [ ]:
plotly.plot(df[['PX_LAST','PX_EMA5','PX_KAMA10']],'line')

### BollingerBands

In [ ]:
BB_obj = ta.volatility.BollingerBands(close=close, window = 20, window_dev = 2, fillna = False)
df.loc[:,'PX_EMA5_BB_hband'] = BB_obj.bollinger_hband()
df.loc[:,'PX_EMA5_BB_lband'] = BB_obj.bollinger_lband()
df.loc[:,'PX_EMA5_BB_pband'] = BB_obj.bollinger_pband()

df[['PX_LAST','PX_EMA5','PX_EMA5_BB_hband','PX_EMA5_BB_lband','PX_EMA5_BB_pband']].head(50)

,PX_LAST,PX_EMA5,PX_EMA5_BB_hband,PX_EMA5_BB_lband,PX_EMA5_BB_pband
Date,,,,,
1990-01-02,585.41,NaN,NaN,NaN,NaN
1990-01-03,585.05,NaN,NaN,NaN,NaN
1990-01-04,580.97,NaN,NaN,NaN,NaN
1990-01-05,575.74,NaN,NaN,NaN,NaN
1990-01-08,577.78,580.024444,NaN,NaN,NaN
1990-01-09,572.00,577.349630,NaN,NaN,NaN
1990-01-10,567.80,574.166420,NaN,NaN,NaN
1990-01-11,569.37,572.567613,NaN,NaN,NaN
1990-01-12,556.16,567.098409,NaN,NaN,NaN


### RSI

In [ ]:
df.loc[:,'PX_EMA5_RSI'] = ta.momentum.rsi(close = close, window=14, fillna=False) 



### MACD

In [ ]:
MACD_obj = ta.trend.MACD(close=close, window_slow =26, window_fast = 12, window_sign = 9, fillna = False)
df.loc[:,'PX_EMA5_MACD'] = MACD_obj.macd()
df.loc[:,'PX_EMA5_MACD_Diff'] = MACD_obj.macd_diff()
df.loc[:,'PX_EMA5_MACD_Sig'] = MACD_obj.macd_signal()


### Volume-price trend (VPT) & Force Index (FI)

In [ ]:
df.loc[:,'PX_EMA5_VPT'] = ta.volume.volume_price_trend(close, volume, fillna=False)
df.loc[:,'PX_EMA5_FI'] = ta.volume.force_index(close, volume, window=13, fillna=False)

### KST Oscillator (KST Signal)

In [ ]:
KST_obj = ta.trend.KSTIndicator(close, roc1 = 10, roc2 = 15, roc3 = 20, roc4 = 30, 
                                window1 = 10, window2 = 10, window3 = 10, window4 = 15, 
                                nsig = 9, fillna = False)
df.loc[:,'PX_EMA5_KST'] = KST_obj.kst()
df.loc[:,'PX_EMA5_KST_Diff'] = KST_obj.kst_diff()
df.loc[:,'PX_EMA5_KST_Sig'] = KST_obj.kst_sig()

In [ ]:
df

,PX_LAST,PX_VOLUME,T1M,T3M,T6M,T1Y,T3Y,T5Y,T10Y,T30Y,T10Y3M,T10Y2Y,LIBOR_3M,TEDRATE,InterestRate,WTI_Futures_Price,WTI_Futures_Return,WTI_Spot_Price,WTI_Spot_Change,Commdity_channel_index_13,Williams_pct_14,RSI_14,Parabolic_0.02_0.2_0.02,DMI_14_ADX,Fear_Greedy_Index,CPI_monthly percent change,CPI_Release,Unemployment_rate,Unemployment_Release,change_in_total_nonfarm_thousand,Nonfarm_Release,VIX_Close,GDP_QOQ_change,GDP_Release,SPY_Adj_Close,SPY_Volume,SP_Adj_Close,SP_Volume,Mkt-RF,SMB,...,month_6,month_7,month_8,month_9,month_10,month_11,month_12,PX_Ret,VIX_Close_Diff,Mkt-RF_Diff,SMB_Diff,HML_Diff,RF_Diff,T1M_Diff,T3M_Diff,T6M_Diff,T1Y_Diff,T3Y_Diff,T5Y_Diff,T10Y_Diff,T30Y_Diff,T10Y3M_Diff,T10Y2Y_Diff,LIBOR_3M_Diff,TEDRATE_Diff,InterestRate_Diff,PX_EMA5,PX_KAMA10,PX_EMA5_BB_hband,PX_EMA5_BB_lband,PX_EMA5_BB_pband,PX_EMA5_RSI,PX_EMA5_MACD,PX_EMA5_MACD_Diff,PX_EMA5_MACD_Sig,PX_EMA5_VPT,PX_EMA5_FI,PX_EMA5_KST,PX_EMA5_KST_Diff,PX_EMA5_KST_Sig
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-02,585.41,NaN,NaN,7.83,7.50,7.81,7.90,7.87,7.94,8.00,0.11,0.07,8.38000,0.80,8.54,22.89,0.0490,22.89,0.0490,158.6294,-0.8405,69.0585,190.157,17.2194,2.3009,0.3,0.0,5.4,0.0,276.0,0.0,17.240000,0.8,0.0,NaN,NaN,359.69,1.620700e+08,1.44,-0.68,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,585.05,NaN,NaN,7.89,7.53,7.85,7.96,7.92,7.99,8.04,0.10,0.05,8.38000,0.75,8.37,23.68,0.0345,23.68,0.0345,166.5505,-6.7745,68.4778,190.767,18.4101,3.0652,0.3,0.0,5.4,0.0,276.0,0.0,18.190001,0.8,0.0,NaN,NaN,358.76,1.923300e+08,-0.06,0.74,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000615,0.950001,-1.50,1.42,-0.19,0.0,NaN,0.06,0.03,0.04,0.06,0.05,0.05,0.04,-0.01,-0.02,0.00000,-0.05,-0.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,580.97,NaN,NaN,7.84,7.50,7.82,7.93,7.91,7.98,8.04,0.14,0.06,8.38000,0.79,8.29,23.41,-0.0114,23.41,-0.0114,104.2355,-19.9820,61.8875,191.575,18.1196,1.8271,0.3,0.0,5.4,0.0,276.0,0.0,19.219999,0.8,0.0,NaN,NaN,355.67,1.770000e+08,-0.71,0.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006998,1.029998,-0.65,-0.32,0.04,0.0,NaN,-0.05,-0.03,-0.03,-0.03,-0.01,-0.01,0.00,0.04,0.01,0.00000,0.04,-0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,575.74,NaN,NaN,7.79,7.46,7.79,7.94,7.92,7.99,8.06,0.20,0.09,8.38000,0.84,8.20,23.08,-0.0141,23.08,-0.0141,56.8323,-36.1185,54.9317,192.319,17.4400,-0.2747,0.3,0.0,5.4,0.0,84.0,1.0,20.110001,0.8,0.0,NaN,NaN,352.20,1.585300e+08,-0.85,0.77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.009043,0.890002,-0.14,0.35,0.01,0.0,NaN,-0.05,-0.04,-0.03,0.01,0.01,0.01,0.02,0.06,0.03,0.00000,0.05,-0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,577.78,NaN,NaN,7.79,7.49,7.81,7.95,7.92,8.02,8.09,0.23,0.12,8.38000,0.84,8.27,21.62,-0.0633,21.62,-0.0633,43.9596,-30.0539,56.8927,193.003,16.6146,-0.5619,0.3,0.0,5.4,0.0,84.0,0.0,20.260000,0.8,0.0,NaN,NaN,353.79,1.401100e+08,0.30,-0.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003537,0.149999,1.15,-1.18,-0.05,0.0,NaN,0.00,0.03,0.02,0.01,0.00,0.03,0.03,0.03,0.03,0.00000,0.00,0.07,580.024444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-19,14547.22,1.322548e+09,0.05,0.17,0.36,0.57,1.33,1.62,1.83,2.14,1.66,0.79,0.25514,0.09,0.08,86.96,0.0179,85.58,-0.0135,-167.3056,-99.3827,32.9727,2780.178,21.6253,-124.8524,0.5,0.0,3.9,0.0,199.0,0.0,23.850000,2.3,0.0,451.750000,109357600.0,4532.76,3.387370e+09,-0.31,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.010162,1.059999,0.00,0.00,0.00,0.0,0.0,0.01,-0.01,-0.01,-0.02,-0.03,-0.04,-0.04,-0.05,-0.02,0.00114,0.01,0.00,14811.

# Decompose Russell 3000 Price

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

## Save CSV

In [ ]:
# output_dir = os.path.join(LABEL_DIR,'master_ds_v4.csv')
# df.to_csv(output_dir)